In [ ]:
print("Testing Notebook") 

Testing Notebook


In [ ]:
# --- In your main.py ---

# (Keep existing imports: base64, UploadFile, time, urllib.parse, etc.)
# (Keep process_files_to_base64_list function as is)

@rt("/send_chat_message")
async def handle_send_chat_message(req, sess): # sess might still be used for other non-attachment things
    form_data = await req.form()
    user_message_text = form_data.get("chatInput", "").strip()
    uploaded_file_objects: list[UploadFile] = form_data.getlist("files")

    attachment_uuids_for_next_step = [] # Store UUIDs of successfully inserted attachments
    processed_attachments_for_preview = [] # For rendering in the user's immediate bubble

    if uploaded_file_objects:
        print(f"Received {len(uploaded_file_objects)} file objects in /send_chat_message.")
        base64_attachments_list = await process_files_to_base64_list(uploaded_file_objects)
        
        if base64_attachments_list:
            processed_attachments_for_preview = base64_attachments_list # For immediate UI
            for att_data in base64_attachments_list:
                if "error" not in att_data and "base64" in att_data:
                    insert_payload = {
                        "base64_string": att_data["base64"],
                        "file_type": att_data.get("content_type"),
                        # Optional: "filename": att_data.get("filename") # if you added filename to your table
                    }
                    try:
                        # Use your initialized supabase client
                        response = supabase.table("akasi_base64_image_strings").insert(insert_payload).execute()
                        if response.data and len(response.data) > 0:
                            new_uuid = response.data[0]['id']
                            attachment_uuids_for_next_step.append(str(new_uuid))
                            print(f"Inserted attachment {att_data.get('filename', 'N/A')} to Supabase with UUID: {new_uuid}")
                        else:
                            print(f"Error inserting {att_data.get('filename', 'N/A')} to Supabase or no data returned. Response: {response}")
                    except Exception as e:
                        print(f"Supabase insert exception for {att_data.get('filename', 'N/A')}: {e}")
                else:
                    print(f"Skipping insert for attachment due to processing error or missing base64: {att_data.get('filename', 'N/A')}")
            print(f"Collected {len(attachment_uuids_for_next_step)} UUIDs from Supabase inserts.")
        else:
            print("No valid attachments processed to store in Supabase.")
    
    if not user_message_text and not processed_attachments_for_preview: # Check based on preview data
        return Textarea(id="chatInput", name="chatInput", placeholder="Describe your symptoms here...", cls="textarea textarea-bordered flex-grow resize-none scrollbar-thin", rows="1", style="min-height: 44px; max-height: 120px;", hx_swap_oob="true")

    # --- Construct User Chat Bubble with Attachment Previews (using processed_attachments_for_preview) ---
    user_chat_bubble_content_elements = []
    if user_message_text:
        user_chat_bubble_content_elements.append(P(user_message_text, cls="text-sm leading-relaxed chat-message-text"))

    if processed_attachments_for_preview:
        attachment_previews_container_items = []
        for att_info in processed_attachments_for_preview: # Use the data we have on hand for preview
            if "error" not in att_info:
                icon_name = "image" if att_info.get("content_type", "").startswith("image/") else "article"
                file_size_kb = (att_info.get('size', 0) / 1024)
                size_text = f"({file_size_kb:.1f} KB)" if file_size_kb > 0 else ""
                
                preview_item = Div(
                    Span(icon_name, cls="material-symbols-outlined emoji-icon text-lg mr-1.5 text-primary/80 flex-shrink-0"),
                    Span(f"{att_info.get('filename', 'N/A')} {size_text}".strip(), cls="text-xs text-base-content/80 truncate"),
                    cls="flex items-center p-1.5 bg-primary/10 rounded"
                )
                attachment_previews_container_items.append(preview_item)
        
        if attachment_previews_container_items:
            user_chat_bubble_content_elements.append(
                Div(*attachment_previews_container_items, cls="mt-2 pt-2 space-y-1.5 border-t border-primary/30")
            )
    
    user_chat_bubble = Div(
        Div(
            Div(
                Div(Span("person", cls="material-symbols-outlined emoji-icon"), cls="bg-transparent text-neutral-content rounded-full w-8 h-8 text-sm flex items-center justify-center"),
                cls="avatar placeholder p-0 w-8 h-8 rounded-full ml-2 user-message-gradient"
            ),
            Div(*user_chat_bubble_content_elements, cls="chat-bubble chat-bubble-primary user-message-gradient rounded-br-none shadow-md"),
            cls="flex items-end max-w-xs sm:max-w-md md:max-w-lg flex-row-reverse"
        ),
        cls="flex justify-end chat-message-container animate-slideUp"
    )

    user_message_text_encoded = urllib.parse.quote(user_message_text)
    
    # Construct attachment_uuids_param
    attachment_uuids_str = ",".join(attachment_uuids_for_next_step)
    attachment_uuids_param = f"&attachment_uuids={attachment_uuids_str}" if attachment_uuids_str else ""

    typing_loader_trigger_id = f"typing-loader-placeholder-{time.time_ns()}"
    typing_loader_trigger = Div(
        id=typing_loader_trigger_id,
        hx_get=f"/load_typing_indicator?user_message={user_message_text_encoded}{attachment_uuids_param}",
        hx_trigger="load delay:500ms", hx_swap="outerHTML"
    )

    cleared_chat_input = Textarea(id="chatInput", name="chatInput", placeholder="Describe your symptoms here...", cls="textarea textarea-bordered flex-grow resize-none scrollbar-thin", rows="1", style="min-height: 44px; max-height: 120px;", hx_swap_oob="true")
 
    return user_chat_bubble, typing_loader_trigger, cleared_chat_input

In [ ]:
# --- In your main.py ---

@rt("/load_typing_indicator")
async def load_typing_indicator_handler(req):
    user_message_text_encoded = req.query_params.get("user_message", "")
    attachment_uuids = req.query_params.get("attachment_uuids", "") # Get the UUIDs string
    ai_bubble_target_id = f"ai-bubble-{time.time_ns()}"

    # Pass attachment_uuids along to the next request
    attachment_uuids_param = f"&attachment_uuids={attachment_uuids}" if attachment_uuids else ""

    typing_indicator_bubble = Div(
        Div( 
            Div(
                Div(Span("smart_toy", cls="material-symbols-outlined emoji-icon"), cls="bg-transparent text-neutral-content rounded-full w-8 h-8 text-sm flex items-center justify-center"),
                cls="avatar placeholder p-0 w-8 h-8 rounded-full mr-2 bg-base-300"
            ),
            Div(P("Akasi is typing...", cls="text-sm italic text-base-content/70 chat-message-text"), cls="chat-bubble chat-bubble-neutral bg-base-300 text-base-content rounded-bl-none shadow-md"),
            cls="flex items-end max-w-xs sm:max-w-md md:max-w-lg" 
        ),
        id=ai_bubble_target_id, 
        cls="flex justify-start chat-message-container animate-slideUp", 
        hx_get=f"/get_ai_actual_response?user_message={user_message_text_encoded}&target_id={ai_bubble_target_id}{attachment_uuids_param}",
        hx_trigger="load delay:50ms", hx_swap="outerHTML"
    )
    return typing_indicator_bubble

In [ ]:
# --- In your main.py ---

@rt("/get_ai_actual_response")
async def get_ai_actual_response(req, sess): # sess might still be used for other non-attachment things
    user_message_text_encoded = req.query_params.get("user_message", "")
    target_id = req.query_params.get("target_id", "") 
    attachment_uuids_str = req.query_params.get("attachment_uuids", "") # Get the UUIDs string

    user_message_text = urllib.parse.unquote(user_message_text_encoded)
    
    retrieved_attachments_for_printing = []
    list_of_uuids_to_delete = [] # Keep track of UUIDs to delete later

    if attachment_uuids_str:
        list_of_uuids = [uid.strip() for uid in attachment_uuids_str.split(',') if uid.strip()]
        list_of_uuids_to_delete = list_of_uuids # Store for deletion
        if list_of_uuids:
            print(f"Attempting to retrieve attachments from Supabase for UUIDs: {list_of_uuids}")
            try:
                # Select specific columns you need, especially base64_string and file_type
                # Add 'filename' if you decided to store it and need it for printing
                response = supabase.table("akasi_base64_image_strings").select("id, base64_string, file_type").in_("id", list_of_uuids).execute()
                
                if response.data:
                    print(f"Retrieved {len(response.data)} attachments from Supabase.")
                    for row in response.data:
                        # Reconstruct a dictionary similar to what process_files_to_base64_list produces
                        # This helps if process_and_print_attachments expects a certain structure
                        retrieved_attachments_for_printing.append({
                            # You might not have the original filename unless you stored it.
                            # For now, we can create a placeholder or use part of the UUID.
                            "filename": f"attachment_{row['id']}.{row['file_type'].split('/')[-1] if row.get('file_type') else 'bin'}",
                            "content_type": row.get('file_type'),
                            "base64": row.get('base64_string'),
                            "size": len(row.get('base64_string', '')) # Approximate size from base64 length
                        })
                else:
                    print(f"No data returned from Supabase for UUIDs: {list_of_uuids}. Response: {response}")
            except Exception as e:
                print(f"Supabase select exception for UUIDs {list_of_uuids}: {e}")
        else:
            print("No valid UUIDs found in attachment_uuids_str.")
            
    # --- Define and call your internal processing function ---
    async def process_and_print_attachments(base64_data_list: list[dict]):
        print("\n--- Entered process_and_print_attachments (in /get_ai_actual_response) ---")
        if not base64_data_list:
            print("No Base64 attachment data received for processing (from Supabase).")
        else:
            for idx, att_data in enumerate(base64_data_list):
                print(f"\nAttachment Content #{idx + 1}:")
                if "error" in att_data: # This 'error' key would be from initial processing if it failed
                     print(f"  Filename: {att_data.get('filename', 'N/A')}")
                     print(f"  Error during previous processing: {att_data['error']}")
                else:
                    print(f"  Filename: {att_data.get('filename', 'N/A')}")
                    print(f"  Content-Type: {att_data.get('content_type', 'N/A')}")
                    print(f"  Size: {att_data.get('size', 0)} bytes (approx. from Base64 length)")
                    base64_snippet = att_data.get('base64', '')[:80] # Snippet
                    print(f"  Base64 Snippet (first 80 chars): {base64_snippet}...")
        print("--- Exited process_and_print_attachments ---\n")

    await process_and_print_attachments(retrieved_attachments_for_printing)
    # --- End of attachment processing and printing ---

    # **Optional but Recommended: Delete from Supabase after processing**
    if list_of_uuids_to_delete:
        try:
            print(f"Attempting to delete processed attachments from Supabase: {list_of_uuids_to_delete}")
            delete_response = supabase.table("akasi_base64_image_strings").delete().in_("id", list_of_uuids_to_delete).execute()
            # Check delete_response for success/failure if needed
            print(f"Supabase delete response: {delete_response}")
        except Exception as e:
            print(f"Supabase delete exception for UUIDs {list_of_uuids_to_delete}: {e}")


    if not user_message_text and not retrieved_attachments_for_printing:
        return Div(
            P("Error: No message or attachments to process.", cls="text-red-500 text-sm"),
            id=target_id, cls="chat-bubble chat-bubble-error"
        )

    llm_output = await llm_agent_1(user_message_text) # Modify llm_agent_1 if it needs attachment data

    ai_chat_bubble_component = unified_ui_controller_for_chat_window_and_body_scanner(
        llm_data=llm_output,
        user_message_text=user_message_text, 
        bubble_id=target_id 
    )

    animation_script_component = None
    scanner_command = llm_output.get('body_scanner_animation_action_comand', "idle")
    if scanner_command and scanner_command.lower() != "idle":
        js_command_call = f"executeBodyScannerCommand('{scanner_command}');"
        animation_script_component = Script(f"(() => {{ try {{ {js_command_call} }} catch (e) {{ console.error('Error executing scanner command:', e); }} }})();")

    if animation_script_component:
        return ai_chat_bubble_component, animation_script_component
    else:
        return ai_chat_bubble_component

# ... (Rest of your main.py: llm_agent_1, UI controllers, other routes, serve())

In [ ]:
# --- In your main.py ---

# (Keep existing imports: asyncio, re, etc.)
# (Keep body_scanner_action_commands_list and pre_configured_conversation_history)

async def llm_agent_1(user_message: str, attachments_data: list[dict] = None): # Added attachments_data parameter
    """
    Simulates an LLM agent processing the user message and attachments.
    Returns a structured response after a delay.
    """
    print(f"\n--- Entered llm_agent_1 ---")
    print(f"llm_agent_1 received user_message: {user_message}")

    # --- Print attachment details if present ---
    if attachments_data:
        print(f"llm_agent_1 received {len(attachments_data)} attachments:")
        for idx, att_data in enumerate(attachments_data):
            print(f"  Attachment #{idx + 1}:")
            if "error" in att_data:
                print(f"    Filename: {att_data.get('filename', 'N/A')}")
                print(f"    Error during previous processing: {att_data['error']}")
            else:
                print(f"    Filename: {att_data.get('filename', 'N/A')}")
                print(f"    Content-Type: {att_data.get('content_type', 'N/A')}")
                print(f"    Size: {att_data.get('size', 0)} bytes (approx. from Base64 length)")
                base64_snippet = att_data.get('base64', '')[:80]  # Snippet
                print(f"    Base64 Snippet (first 80 chars): {base64_snippet}...")
    else:
        print("llm_agent_1 received no attachments.")
    # --- End of attachment printing ---

    await asyncio.sleep(2) # Simulate processing delay

    found_command = "idle" # Default command
    normalized_user_message = user_message.lower()

    # (Your existing command detection logic remains the same)
    if "start scan" in normalized_user_message:
        found_command = "START_SCAN"
    elif "stop scan" in normalized_user_message:
        found_command = "STOP_SCAN"
    elif "full body glow" in normalized_user_message or "glow" in normalized_user_message:
        found_command = "FULL_BODY_GLOW"
    else:
        for command_keyword in body_scanner_action_commands_list:
            if command_keyword not in ["START_SCAN", "STOP_SCAN", "FULL_BODY_GLOW"]:
                if re.search(r'\b' + re.escape(command_keyword.lower()) + r'\b', normalized_user_message):
                    found_command = command_keyword
                    break
    
    response_data = {
        "ai_response": f"Simulated AI response to: '{user_message}'. Action: {found_command}",
        "body_scanner_animation_action_comand": found_command,
        "trigger_wellness_journal_data_listener": "START",
        "output_conversation_history": pre_configured_conversation_history
    }
    print(f"llm_agent_1 generated output: {response_data}")
    print(f"--- Exited llm_agent_1 ---\n")
    return response_data

In [ ]:
# --- In your main.py ---

# (Keep existing imports: urllib.parse, supabase client, etc.)

@rt("/get_ai_actual_response")
async def get_ai_actual_response(req, sess): # sess might still be used for other non-attachment things
    user_message_text_encoded = req.query_params.get("user_message", "")
    target_id = req.query_params.get("target_id", "") 
    attachment_uuids_str = req.query_params.get("attachment_uuids", "")

    user_message_text = urllib.parse.unquote(user_message_text_encoded)
    
    retrieved_attachments_from_supabase = [] # Renamed for clarity
    list_of_uuids_to_delete = []

    if attachment_uuids_str:
        list_of_uuids = [uid.strip() for uid in attachment_uuids_str.split(',') if uid.strip()]
        list_of_uuids_to_delete = list_of_uuids
        if list_of_uuids:
            print(f"Attempting to retrieve attachments from Supabase for UUIDs: {list_of_uuids}")
            try:
                response = supabase.table("akasi_base64_image_strings").select("id, base64_string, file_type").in_("id", list_of_uuids).execute()
                
                if response.data:
                    print(f"Retrieved {len(response.data)} attachments from Supabase.")
                    for row in response.data:
                        retrieved_attachments_from_supabase.append({
                            "filename": f"attachment_{row['id']}.{row['file_type'].split('/')[-1] if row.get('file_type') else 'bin'}",
                            "content_type": row.get('file_type'),
                            "base64": row.get('base64_string'),
                            "size": len(row.get('base64_string', ''))
                        })
                else:
                    print(f"No data returned from Supabase for UUIDs: {list_of_uuids}. Response: {response}")
            except Exception as e:
                print(f"Supabase select exception for UUIDs {list_of_uuids}: {e}")
        else:
            print("No valid UUIDs found in attachment_uuids_str.")
            
    # --- REMOVE the old process_and_print_attachments function definition and its call ---
    # async def process_and_print_attachments(base64_data_list: list[dict]): ...
    # await process_and_print_attachments(retrieved_attachments_from_supabase) # This call is removed

    # **Optional but Recommended: Delete from Supabase after retrieval (before passing to LLM)**
    # It's better to delete after you are sure the LLM (or subsequent steps) won't need to re-fetch.
    # For now, we'll keep the deletion logic here. If llm_agent_1 fails, data might be lost.
    # Consider moving deletion to after llm_agent_1 successfully processes, or implement a retry/dead-letter queue.
    if list_of_uuids_to_delete:
        try:
            print(f"Attempting to delete processed attachments from Supabase: {list_of_uuids_to_delete}")
            delete_response = supabase.table("akasi_base64_image_strings").delete().in_("id", list_of_uuids_to_delete).execute()
            print(f"Supabase delete response: {delete_response}")
        except Exception as e:
            print(f"Supabase delete exception for UUIDs {list_of_uuids_to_delete}: {e}")

    if not user_message_text and not retrieved_attachments_from_supabase:
        return Div(
            P("Error: No message or attachments to process.", cls="text-red-500 text-sm"),
            id=target_id, cls="chat-bubble chat-bubble-error"
        )

    # --- Call llm_agent_1, passing the retrieved attachments ---
    llm_output = await llm_agent_1(user_message_text, attachments_data=retrieved_attachments_from_supabase) 

    ai_chat_bubble_component = unified_ui_controller_for_chat_window_and_body_scanner(
        llm_data=llm_output,
        user_message_text=user_message_text, 
        bubble_id=target_id 
    )

    animation_script_component = None
    scanner_command = llm_output.get('body_scanner_animation_action_comand', "idle")
    if scanner_command and scanner_command.lower() != "idle":
        js_command_call = f"executeBodyScannerCommand('{scanner_command}');"
        animation_script_component = Script(f"(() => {{ try {{ {js_command_call} }} catch (e) {{ console.error('Error executing scanner command:', e); }} }})();")

    if animation_script_component:
        return ai_chat_bubble_component, animation_script_component
    else:
        return ai_chat_bubble_component

# ... (Rest of your main.py)